In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
#print(torch.cuda.get_device_name(0))

In [3]:
!pip install transformers sentencepiece accelerate
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

## 載入資料集與模型

In [4]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# 載入資料集
dataset = load_dataset("Maciel/FinCUGE-Instruction")

# 載入預訓練的中文 BERT 模型與 tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=3)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.86k [00:00<?, ?B/s]

(…)-00000-of-00001-6150fcadf0837f0f.parquet:   0%|          | 0.00/28.6M [00:00<?, ?B/s]

(…)-00000-of-00001-fe302d3367c7457a.parquet:   0%|          | 0.00/3.44M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/123137 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


用 Dataset.filter() 清除無效樣本

In [5]:
from datasets import DatasetDict

dataset_org = dataset
# 使用 论坛情绪分析任务
filtered_dataset = dataset_org.filter(lambda example: example['desc'] == '论坛情绪分析任务')
dataset = filtered_dataset
# 顯示筆數
print(f"Train before :{len(dataset_org['train'])}, after :{len(dataset['train'])}")
print(f"Eval before :{len(dataset_org['eval'])}, after :{len(dataset['eval'])}")


Filter:   0%|          | 0/123137 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15167 [00:00<?, ? examples/s]

Train before :123137, after :16157
Eval before :15167, after :2020


從自然語言中抽出情緒詞

In [6]:
import re

label_map = {
    "积极": 0,
    "消极": 1,
    "中性": 2
}
def extract_label(output_str):
    # 簡單的方式：直接尋找關鍵詞
    for key in label_map:
        if key in output_str:
            return label_map[key]
    # 找不到就視為無效
    raise ValueError(f"無法從 output 中解析出標籤: {output_str}")

資料預處理

In [7]:
'''#acc : 0.78
def preprocess_function(examples):
    tokenized = tokenizer(
        examples['input'],
        truncation=True,
        padding='max_length',
        max_length=128
    )
    #抓出情緒詞
    tokenized["labels"] = [extract_label(o) for o in examples["output"]]
    return tokenized
'''
def preprocess_function(examples):
    texts = [i + " " + j for i, j in zip(examples["instruction"], examples["input"])]
    tokenized = tokenizer(texts, truncation=True, padding='max_length', max_length=128)
    tokenized["labels"] = [extract_label(o) for o in examples["output"]]
    return tokenized


tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/16157 [00:00<?, ? examples/s]

Map:   0%|          | 0/2020 [00:00<?, ? examples/s]

訓練模型

In [8]:
import os
from transformers import EarlyStoppingCallback
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    logging_dir="./logs",
    learning_rate=1e-5,  #LR
    warmup_steps=500,   #LR
    load_best_model_at_end=True,#Early Stop
    save_strategy="epoch",
)

#驗證指標
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='macro')
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] #EARLY STOP
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.676900,0.598626,0.749505,0.725249
2,0.537100,0.565114,0.782178,0.763387
3,0.425100,0.589307,0.787624,0.771851
4,0.333000,0.631710,0.793069,0.781595
5,0.257700,0.769117,0.792079,0.776003


TrainOutput(global_step=5050, training_loss=0.46926753185763215, metrics={'train_runtime': 1917.3927, 'train_samples_per_second': 84.265, 'train_steps_per_second': 5.268, 'total_flos': 5313904362789120.0, 'train_loss': 0.46926753185763215, 'epoch': 5.0})

In [9]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_path = f"./results/model_{timestamp}"
os.makedirs(model_path, exist_ok=True)

trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)
trainer.save_state()

In [10]:
timestamp

'20250531_030542'

In [11]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# 載入 tokenizer 和 微調後的模型
model_path = f"./results/model_{timestamp}"  # 假設你存在這個目錄
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)
model.eval()  # 設定為推論模式

# 定義標籤對應（你訓練時應該是三類情感）
label_map = {0: "消極", 1: "中性", 2: "積極"}

text = "這支股票今天開低走高，明天應該會繼續漲！"

# 前處理
input_text = "这个文本的情感倾向是积极、消极还是中性的。 " + text  # 與訓練時格式一致
# Tokenize 輸入
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=128)



# 預測
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()
    predicted_label = label_map[predicted_class_id]

# 輸出結果
print(f"輸入句子：{text}")
print(f"預測情感：{predicted_label}")


輸入句子：這支股票今天開低走高，明天應該會繼續漲！
預測情感：消極


In [12]:
#rm -r results

類別平衡?

In [13]:
from collections import Counter
Counter(tokenized_datasets["train"]["labels"])

Counter({2: 4143, 1: 4690, 0: 7324})

CONFUSION MATRIX

In [14]:
from sklearn.metrics import confusion_matrix
preds = trainer.predict(tokenized_datasets["eval"]).predictions.argmax(axis=1)
labels = tokenized_datasets["eval"]["labels"]
print(confusion_matrix(labels, preds))


[[780  45  61]
 [ 35 472  67]
 [136  96 328]]
